# BM Reports API Example

In order to use this code you will need to go onto the ELEXON Portal website [click here](https://www.elexonportal.co.uk) to register and get yourself an API key if you want to download the data yourself. The guide to using the API can be found by [clicking here](https://www.elexon.co.uk/guidance-note/bmrs-api-data-push-user-guide/). From this you should be able to work out the arguments that need to be passed to the functions below.

First need some stuff to query a website and deal with xml data types.

In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import xmltodict

Use a function to build the request string to the API website and deliver the requested xml file into a variable on return.

In [2]:
def BMRS_GetXML(**kwargs):
    '''BMRS_XMLGet(api='###', report='PHYBMDATA', sd='2016-01-26', sp=3,
    bmu='T_COTPS-1',bmutype='T', leadpartyname = 'AES New Energy Limited',ngcbmuname='EAS-ASP01')'''

    url = 'https://api.bmreports.com/BMRS/{report}/v1?APIKey={api}&ServiceType=xml'.format(**kwargs)

    for key, value in kwargs.items():
        if key not in ['report','api']:
            a = "&%s=%s" % (key, value)
            url = url + a
    
    # print url to screen to check request
    print(url)
    response = requests.get(url)
    xml =  ET.fromstring(response.text)
    return xml


Using the xml file, we can easily turn this into a pandas dataframe with the following function:

In [3]:
def BMRS_Dataframe(**kwargs):
    '''Takes the sourced XML file produces a dataframe from all the children of the ITEM tag'''
    tags = []

    # get the xml file
    xml = BMRS_GetXML(**kwargs)
    
    # get tags, select first item to get tags
    root = xml.find("./responseBody/responseList/item")
    for child in root:
        tags.append(child.tag)
    
    df = pd.DataFrame(columns=tags)

    for root in xml.findall("./responseBody/responseList/item"):
        tags = []

        rowData = xmltodict.parse(ET.tostring(root))
      
        df = df.append(rowData['item'], ignore_index=True)
        
    return df

Now we can build up a call to this function, including the required arguments for the particular request we want. In this case we take the system data file (called `DERSYSDATA`) that has information from the balancing market.

In [4]:
# get the report on system buy/sell prices
# YOU NEED TO GET YOUR OWN API KEY TO SUB IN HERE!!!!
APIKEY='###################'
REPORT='DERSYSDATA'
FROMSETTLEMENTDATE='2020-04-01'
TOSETTLEMENTDATE='2020-04-03'
SETTLEMENTPERIOD='*'
df = BMRS_Dataframe(api=APIKEY,report=REPORT, FromSettlementDate=FROMSETTLEMENTDATE,ToSettlementDate=TOSETTLEMENTDATE,SettlementPeriod=SETTLEMENTPERIOD)

https://api.bmreports.com/BMRS/DERSYSDATA/v1?APIKey=o1ne017ua1efj1w&ServiceType=xml&FromSettlementDate=2020-04-01&ToSettlementDate=2020-04-03&SettlementPeriod=*


Some post processing converts the columns into appropriate data types (by default they are all `string`s), and creates a new column combining the Settlement Date and Period into a `datetime` variable.

In [5]:
# adjust the columns to appropriate types

df["settlementDate"]=pd.to_datetime(df["settlementDate"])
df["settlementPeriod"]=pd.to_numeric(df["settlementPeriod"])
df["systemSellPrice"]= df["systemSellPrice"].astype(float)
df["systemBuyPrice"]= df["systemBuyPrice"].astype(float)
df["indicativeNetImbalanceVolume"]= df["indicativeNetImbalanceVolume"].astype(float)

# make a datetime version by adding on the half hour periods

df['settlementDateTime'] = df['settlementDate'] + pd.to_timedelta( ( (df.settlementPeriod-1)/2.) , unit='h')

It looks like this:

In [6]:
df

,recordType,settlementDate,settlementPeriod,systemSellPrice,systemBuyPrice,bSADDefault,priceDerivationCode,reserveScarcityPrice,indicativeNetImbalanceVolume,sellPriceAdjustment,...,totalSystemTaggedAcceptedOfferVolume,totalSystemTaggedAcceptedBidVolume,totalSystemAdjustmentSellVolume,totalSystemAdjustmentBuyVolume,totalSystemTaggedAdjustmentSellVolume,totalSystemTaggedAdjustmentBuyVolume,activeFlag,replacementPrice,replacementPriceCalculationVolume,settlementDateTime
0,SSB,2020-04-01,1,38.75,38.75,F,P,NULL,428.1275,0.00,...,929.327,-242.700,-357.000,97.500,-357.000,97.500,Y,NaN,NaN,2020-04-01 00:00:00
1,SSB,2020-04-01,2,38.75,38.75,F,P,NULL,153.5157,0.00,...,715.286,-303.270,-357.000,97.500,-357.000,97.500,Y,NaN,NaN,2020-04-01 00:30:00
2,SSB,2020-04-01,3,38.75,38.75,F,P,NULL,181.7627,0.00,...,762.600,-322.337,-357.000,97.500,-357.000,97.500,Y,NaN,NaN,2020-04-01 01:00:00
3,SSB,2020-04-01,4,38.80,38.80,F,P,NULL,399.1462,0.00,...,985.567,-327.913,-357.000,97.500,-357.000,97.500,Y,NaN,NaN,2020-04-01 01:30:00
4,SSB,2020-04-01,5,38.80,38.80,F,P,NULL,402.9604,0.00,...,1049.360,-417.867,-327.000,97.500,-327.000,97.500,Y,NaN,NaN,2020-04-01 02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,SSB,2020-04-03,44,10.00,10.00,F,N,NULL,-541.6782,0.00,...,385.275,-925.953,0.000,0.000,0.000,0.000,Y,NaN,NaN,2020-04-03 21:30:00
140,SSB,2020-04-03,45,10.00,10.00,F,N,NULL,-405.4580,0.00,...,421.542,-826.000,0.000,0.000,0.000,0.000,Y,NaN,NaN,2020-04-03 22:00:00
141,SSB,2020-04-03,46,10.00,10.00,F,N,NULL,-277.0777,0.00,...,595.065,-871.143,0.000,0.000,0.000,0.000,Y,NaN,NaN,2020-04-03 22:30:00
142,SSB,2020-04-03,47,36.50,36.50,F,P,NULL,5.6952,0.00,...,965.037,-547.320,-413.000,0.000,-413.000,0.000,Y,NaN,NaN,2020-04-03 23:00:00


You can then output this into a csv file that can be opened in excel or imported into other programming languages.

In [7]:
# output file to csv to open in excel (or other programming language)
df.to_csv("test.csv")

Another example below, reading off the market price from the report `MID`.

In [8]:
# YOU NEED TO GET YOUR OWN API KEY TO SUB IN HERE!!!!
APIKEY='####################'
REPORT='MID'
FROMSETTLEMENTDATE='2020-04-01'
TOSETTLEMENTDATE='2020-04-03'
PERIOD='*'
dfMID = BMRS_Dataframe(api=APIKEY,report=REPORT, FromSettlementDate=FROMSETTLEMENTDATE,ToSettlementDate=TOSETTLEMENTDATE,Period=PERIOD)

https://api.bmreports.com/BMRS/MID/v1?APIKey=o1ne017ua1efj1w&ServiceType=xml&FromSettlementDate=2020-04-01&ToSettlementDate=2020-04-03&Period=*


In [9]:
# adjust the columns to appropriate types
dfMID["settlementDate"]=pd.to_datetime(dfMID["settlementDate"])
dfMID["settlementPeriod"]=pd.to_numeric(dfMID["settlementPeriod"])
dfMID["marketIndexPrice"]= dfMID["marketIndexPrice"].astype(float)
dfMID["marketIndexVolume"]= dfMID["marketIndexVolume"].astype(float)

# make a datetime version by adding on the half hour periods
dfMID['settlementDateTime'] = dfMID['settlementDate'] + pd.to_timedelta( ( (dfMID.settlementPeriod-1)/2.) , unit='h')

In [10]:
# select only APXMIDP prices
dfMID[dfMID.marketIndexDataProviderId=="APXMIDP"]

,recordType,marketIndexDataProviderId,settlementDate,settlementPeriod,marketIndexPrice,marketIndexVolume,activeFlag,settlementDateTime
0,MID,APXMIDP,2020-04-01,1,36.23,296.25,Y,2020-04-01 00:00:00
1,MID,APXMIDP,2020-04-01,2,35.42,223.80,Y,2020-04-01 00:30:00
2,MID,APXMIDP,2020-04-01,3,33.92,434.55,Y,2020-04-01 01:00:00
3,MID,APXMIDP,2020-04-01,4,35.71,466.10,Y,2020-04-01 01:30:00
4,MID,APXMIDP,2020-04-01,5,36.20,592.95,Y,2020-04-01 02:00:00
...,...,...,...,...,...,...,...,...
139,MID,APXMIDP,2020-04-03,44,27.90,1572.90,Y,2020-04-03 21:30:00
140,MID,APXMIDP,2020-04-03,45,26.71,1353.65,Y,2020-04-03 22:00:00
141,MID,APXMIDP,2020-04-03,46,23.47,1852.75,Y,2020-04-03 22:30:00
142,MID,APXMIDP,2020-04-03,47,15.27,759.85,Y,2020-04-03 23:00:00
